# Step 0: Defining Model Class

In [ ]:
# Basics
import pandas as pd
import numpy as np

#Plotting
from matplotlib import pyplot as plt
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
plt.style.use('ggplot')

# Warnings
import warnings
warnings.filterwarnings('ignore')

#Auto ARIMA
try:
    from pmdarima.arima import auto_arima
except:
    ! pip install pmdarima
    from pmdarima.arima import auto_arima

class Model(object):
    
    def __init__(self, test_size=48, predict_size=24):
        self.model_dictionary = {}
        self.test_size = test_size
        self.predict_size = predict_size
        
    def get_zip_code(self, row):
        return row['RegionName']
    
    def make_datetime_index(self, df):
        string_index = df.columns.values[8:]
        self.datetime_index = pd.to_datetime(string_index)
    
    def load_data(self, df):
        self.make_datetime_index(df)
        for ind in df.index.values:
            row = df.loc[ind]
            zip_code = self.get_zip_code(row)
            row_dict = self.make_row_dict(row)
            self.model_dictionary[zip_code] = row_dict
        
    def make_time_series_df(self, time_series):
        time_series.index = self.datetime_index
        df = pd.DataFrame(time_series)
        df.columns = ['MedianSales']
        return df
        
    def make_row_dict(self, row):
        row_dict = row.iloc[1:8].to_dict()
        time_series = row.iloc[8:]
        df = self.make_time_series_df(time_series)
        df = df.fillna(df.bfill())
        row_dict['TimeSeries'] = df
        return row_dict
            
    def get_time_series(self, zip_code):
        return self.model_dictionary[zip_code]['TimeSeries']
        
    def make_train_test_split(self, zip_code):
        time_series_df = self.get_time_series(zip_code)
        train = time_series_df[:-self.test_size]
        test = time_series_df[-self.test_size:]
        self.model_dictionary[zip_code]['TrainSeries'] = train
        self.model_dictionary[zip_code]['TestSeries'] = test
        
    def get_train_series(self, zip_code):
        try:
            train_series = self.model_dictionary[zip_code]['TrainSeries']
        except:
            self.make_train_test_split(zip_code)
            train_series = self.model_dictionary[zip_code]['TrainSeries']
        return train_series
            
    def get_test_series(self, zip_code):
        try:
            test_series = self.model_dictionary[zip_code]['TestSeries']
        except:
            self.make_train_test_split(zip_code)
            test_series = self.model_dictionary[zip_code]['TestSeries']
        return test_series
        
    def get_city_name(self, zip_code):
        return self.model_dictionary[zip_code]['City']
    
    def get_state_abbreviation(self, zip_code):
        return self.model_dictionary[zip_code]['State']
    
    def make_time_series_plot(self, zip_code):
        train = self.get_train_series(zip_code)
        test = self.get_test_series(zip_code)
        city_name = self.get_city_name(zip_code)
        state_abbreviation = self.get_state_abbreviation(zip_code)
        fig, ax = plt.subplots(figsize=(8, 5));
        ax.plot(train);
        ax.plot(test);
        legend_labels = ['Train', 'Test']
        try:
            in_sample_prediction = self.get_in_sample_prediction(zip_code)
            ax.plot(in_sample_prediction)
            legend_labels.append('Prediction')
        except:
            pass
        ax.legend(legend_labels);
        ax.set_title(f'Median Sale Price {city_name}, {state_abbreviation} {zip_code}')
        ax.set_xlabel('Date')
        ax.set_ylabel('Price')
        return fig
    
    def make_acf_plot(self, zip_code):
        city_name = self.get_city_name(zip_code)
        state_abbreviation = self.get_state_abbreviation(zip_code)
        train = self.get_train_series(zip_code)
        test = self.get_test_series(zip_code)
        fig, ax = plt.subplots(figsize=(8,5));
        plot_acf(
            x=train, 
            ax=ax, 
            lags=100,
            title=f'Autocorrelation for {city_name}, {state_abbreviation} {zip_code}'
        );
        return fig
    
    def make_pacf_plot(self, zip_code):
        city_name = self.get_city_name(zip_code)
        state_abbreviation = self.get_state_abbreviation(zip_code)
        train = self.get_train_series(zip_code)
        test = self.get_test_series(zip_code)
        fig, ax = plt.subplots(figsize=(8,5));
        plot_pacf(
            x=train, 
            ax=ax, 
            lags=100,
            title=f'Partial Autocorrelation for {city_name}, {state_abbreviation} {zip_code}'
        );
        return fig
    
    def select_model(self, zip_code, trace=True):
        y = self.get_time_series(zip_code)
        model = auto_arima(
            y = y,
            X=None,
            start_p=0,
            d=1,
            start_q=0,
            max_p=2,
            max_d=2,
            max_q=2,
            start_P=0,
            D=1,
            start_Q=0,
            max_P=2,
            max_D=2,
            max_Q=2,
            max_order=None,
            m=12,
            seasonal=True,
            stationary=False,
            information_criterion='oob',
            alpha=0.05,
            test='kpss',
            seasonal_test='OCSB',
            stepwise=True,            
            suppress_warnings=True,
            error_action='warn',
            trace=trace,
            out_of_sample_size= self.test_size,
            scoring='mse'
        )
        self.model_dictionary[zip_code]['BestModel'] = model
    
    def get_best_model(self, zip_code):
        return self.model_dictionary[zip_code]['BestModel']
            
    def predict_in_sample(self, zip_code):
        model = self.get_best_model(zip_code)
        time_series = self.get_time_series(zip_code)
        index = time_series.index
        in_sample_prediction = model.predict_in_sample()
        in_sample_prediction_df = pd.DataFrame(in_sample_prediction, index=index)
        in_sample_prediction_df.columns = ['MedianSales']
        self.model_dictionary[zip_code]['InSamplePrediction'] = in_sample_prediction_df
            
    def get_in_sample_prediction(self, zip_code):
        return self.model_dictionary[zip_code]['InSamplePrediction']
    
    def predict(self, zip_code):
        model = self.get_best_model(zip_code)
        prediction = model.predict(n_periods=self.predict_size)
        return prediction


In [ ]:
pd.date_range()

# Step 1: Import the Data
Our main data set is stored in the `zillow_data.csv` spreadsheet. We load the CSV file as a data frame below.

In [ ]:
df = pd.read_csv('../data/Zip_zhvi_uc_sfr_tier_0.33_0.67_sm_sa_mon.csv', index_col='RegionID')
print(df.shape)
df.head()

We will restrict our attention to the city of Baltimore, so we filter the data below.

In [ ]:
df = df.query("City == 'Baltimore' and State == 'MD'")
print(df.shape)
df.head()

# Step 2: Instantiate the Model and Load in the Data
Below we instantiate a Model object and load our selected data into the object.

In [ ]:
model = Model()
model.load_data(df)

In [ ]:
model.make_predict_index()

# Step 3: EDA and Visualization
Below we display a time series plot, ACF plot, and PACF plot for the zip code 21215. The class methods used below will work for any zip code in our model.

In [ ]:
model.make_time_series_plot(21215);

In [ ]:
model.make_acf_plot(21215);

In [ ]:
model.make_pacf_plot(21215);

# Step 4: ARIMA Modeling

In [ ]:
model.select_model(21215)

# Step 5: Interpreting Results

In [ ]:
model.predict_in_sample(21215)

In [ ]:
model.make_time_series_plot(21215);

In [ ]:
best_model = model.get_best_model(21215)

In [ ]:
pred = best_model.predict(n_periods=)

In [ ]:
pred[1]

In [ ]:
import sys
sys.getsizeof(model.model_dictionary)